### Importing libraries

In [1]:
import pandas as pd
import re

### Reading database files

In [2]:
customers = pd.read_csv('Data/customers.csv')
invoice_items = pd.read_csv('Data/invoice_items.csv')
invoice = pd.read_csv('Data/invoice.csv')
#product_categories = pd.read_csv('Data/product_categories.csv')
product_items = pd.read_csv('Data/product_items.csv')
stores = pd.read_csv('Data/stores.csv')

### Parsing and merging the data

In [3]:
product_items['category_id'].replace({1:'Home',
                                      2:'Arms',
                                      3:'Books',
                                      4:'Industrial',
                                      5:'Grocery',
                                      6:'Computers',
                                      7:'Shoes',
                                      8:'Men Clothing',
                                      9:'Electronics',
                                      10:'Baby',
                                      11:'Toys',
                                      12:'Games',
                                      13:'Outdoors',
                                      14:'Woman Clothing',
                                      15:'Kids'}, inplace=True)

In [4]:
df = pd.merge(stores, invoice, on='store_id')
df.drop(columns=['customer_id', 'store_id', 'invoice_date'], inplace=True)
df = pd.merge(df, invoice, on='invoice_id')
df = pd.merge(df, invoice_items, on='invoice_id')
df = pd.merge(df, product_items, on='item_id')
df = pd.merge(df, customers, on='customer_id')

df[['invoice_date', 'invoice_hour']] = df.invoice_date.str.split(" ", expand=True)
#df.drop(columns=['invoice_date'], inplace=True)

In [12]:
df = df[['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id', 'store_name', 'store_location_lat', 'store_location_long',
         'customer_name', 'customer_lastname', 'customer_phone', 'customer_email', 'category_id', 'product_name', 'product_price',
         'quantity', 'invoice_date', 'invoice_hour']]
df.shape

(2000, 18)

## Ventas por hora de la tienda seleccionada en un tiempo determinado

In [6]:
def find_sum(str1):
    return sum(map(int, re.findall('\d+', str1)))

fecha_inicio = '2021-01-25'
fecha_fin = '2021-01-25'
hora_inicio = '18:00:00'
hora_fin = '20:39:32'

df['invoice_date_sum'] = df['invoice_date'].apply(lambda x: sum(map(int, re.findall('\d+', x))))
df['invoice_hour_sum'] = df['invoice_hour'].apply(lambda x: sum(map(int, re.findall('\d+', x))))

df_time = df.loc[(df['invoice_date_sum']>=find_sum(fecha_inicio)) & (df['invoice_hour_sum']>=find_sum(hora_inicio)) & 
                 (df['invoice_date_sum']<=find_sum(fecha_fin)) & (df['invoice_hour_sum']<=find_sum(hora_fin))]

df_time = df_time.groupby(by='store_name').sum()
df_time.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                      'store_location_lat', 'store_location_long', 'invoice_date_sum', 'invoice_hour_sum'], inplace=True)
df_time.head(10)

,product_price,quantity
store_name,,
AC CONSEQUAT Store,172.72,36
AUGUE LUCTUS Store,195.95,25
CONGUE Store,125.67,4
CURAE Store,163.59,15
LIBERO Store,172.85,31
MAECENAS Store,152.91,12
NAM Store,227.85,10
NULLA Store,147.16,14
ORCI LUCTUS Store,207.37,15


## Top 20 productos más vendidos en total y por tienda.

##### Productos más vendidos en total

In [9]:
Productos_mas_vendidos = df.groupby(by='product_name').sum()
Productos_mas_vendidos = Productos_mas_vendidos.sort_values('quantity', ascending=False)
Productos_mas_vendidos.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                                     'store_location_lat', 'store_location_long'], inplace=True)
Productos_mas_vendidos.head(20)

,product_price,quantity,invoice_date_sum,invoice_hour_sum
product_name,,,,
Cod - Black Whole Fillet,1682.32,210,69354,2327
Wine - Jackson Triggs Okonagan,866.13,180,104145,3721
"Shrimp - 16/20, Peeled Deviened",1384.50,160,50981,1773
Venison - Racks Frenched,2220.24,158,49061,1690
Lettuce - Red Leaf,1961.50,155,51079,1788
Jolt Cola,244.62,153,55127,2080
Kellogs All Bran Bars,637.20,153,48989,1651
V8 Splash Strawberry Kiwi,2180.88,150,48921,1677
"Cheese - Cheddar, Mild",1798.32,150,49005,1753


##### Productos más vendidos en total y por tienda

In [11]:
Productos_mas_vendidos_tienda = df.groupby(by=['store_name', 'product_name']).sum()
Productos_mas_vendidos_tienda = Productos_mas_vendidos_tienda.sort_values(['store_name', 'quantity'], ascending=False)
Productos_mas_vendidos_tienda.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                                     'store_location_lat', 'store_location_long'], inplace=True)
Productos_mas_vendidos_tienda

product_price  quantity  \
store_name          product_name                                              
VOLUTPAT QUAM Store Lemonade - Mandarin, 591 Ml            205.36        13   
                    Nut - Pistachio, Shelled               141.03        13   
                    Wine - Jackson Triggs Okonagan           4.17        12   
                    Ecolab Crystal Fusion                    8.72        11   
                    Milk 2% 500 Ml                         181.78        10   
...                                                           ...       ...   
A Store             Swiss Chard - Red                       27.44         2   
                    Chip - Potato Dill Pickle               85.85         1   
                    Molasses - Fancy                        63.42         1   
                    Pimento - Canned                         8.34         1   
                    Wine - Penfolds Koonuga Hill            19.50         1   

                                                    invoice_date_sum  \
store_name          product_name                                       
VOLUTPAT QUAM Store Lemonade - Mandarin, 591 Ml                 8171   
                    Nut - Pistachio, Shelled                    6129   
                    Wine - Jackson Triggs Okonagan              6110   
                    Ecolab Crystal Fusion                       8163   
                    Milk 2% 500 Ml                              4100   
...                                                              ...   
A Store             Swiss Chard - Red                           2054   
                    Chip - Potato Dill Pickle                   2047   
                    Molasses - Fancy                            2045   
                    Pimento - Canned                            2033   
                    Wine - Penfolds Koonuga Hill                2037   

                                                    invoice_hour_sum  
store_name          product_name                                      
VOLUTPAT QUAM Store Lemonade - Mandarin, 591 Ml                  380  
                    Nut - Pistachio, Shelled                     240  
                    Wine - Jackson Triggs Okonagan               205  
                    Ecolab Crystal Fusion                        377  
                    Milk 2% 500 Ml                               159  
...                                                              ...  
A Store             Swiss Chard - Red                             62  
                    Chip - Potato Dill Pickle                    106  
                    Molasses - Fancy                              25  
                    Pimento - Canned                              49  
                    Wine - Penfolds Koonuga Hill                  85  

[972 rows x 4 columns]

## Preguntas adicionales

#### Una imagen o pdf con el Diagrama de entidades y relaciones

![Relaciones de las tablas](Assets/tables-relations.png "Relaciones de las tablas")


#### Imaginen esta BD transaccional que contiene los datos de 1000 tiendas para cubrir una población de 30millones de personas con toda su historia de ventas de 2 años, ¿Qué sistema de almacenamiento recomendarían y que métodos de optimización?

Como sistema de almacenamiento se podría usar Hadoop con el fin de almacenar todos esos datos de las 1000 tiendas de manera distribuida y así facilitar su consulta. Como método de optimización se podría usar el framework de Hive ya que permite agrupar, consultar y analizar datos mediante un sistema basado en queries de SQL llamado Hive QL y este mismo aplica un mapReduce.

#### Imaginen que tienen acceso a cualquier tecnología “state-of-the-art”, y requieren crear un sistema analítico para poder obtener todo tipo de estudios de ciencia de datos con los datos de la BD, ¿Qué tecnologías recomendarían?, no solo para el almacenamiento de datos sino el procesamiento de máquina y la visualización que permita crear y compartir dashboards que puedan mostrar los KPI’s en tiempo real o interactivos.

Para el almacenamiento de los datos se usaria una base de datos relacional y para el procesamiento y visualización de datos se podría usar la nube de Azure ya que ofrece bastantes modelos de AI para hacer diferentes analisis y predicciones junto con bastantes opciones de visualización y procesamiento de datos en tiempo real.

#### Basado en los datos de la BD diseñada, ¿Qué método usaría para poder hacer un forecast de las ventas para cualquiera de las tiendas de algún día del futuro seleccionado?

Basado en el comportamiento que tuvieron las ventas del 2020 y 2021 todas las tiendas se adaptaría mejor un modelo de alisado exponencial simple por su facilidad de desarrollar, implementar y el comportamiento tan disparejo de los datos de cada tienda.

#### En caso necesario de compartir un dashboard PBI a una audiencia de 700 personas, ¿Qué haría o recomendaría hacer?

Poder compartir el reporte exportando el reporte en formato de HTML para que no tengan problemas de compatibilidad con ningún dispositivo u otra alternativa sería publicar el mismo reporte a la web para que los usuarios solo tengan que acceder a un link para ver e interactuar con el reporte.

#### Imagine que además de tener la información de ventas, también tiene un identificador único de cada cliente de cada tiquete de compra en su base de datos, ¿Qué tipo de estudio o análisis haría?

Analizar los productos que más compra cada cliente con el fin de maximizar el stock de cada tienda y reducir costos de productos que no se venden. Adicionalmente se podría analizar los productos y tendencias de compra de cada persona con el fin de personalizar la publicidad y ventas a los clientes.

#### Procedimiento creación de la base de datos

1. Primero se empezó diseñando los campos y relaciones que iba a tener la base de datos.
2. Luego se usó [mockaroo](https://mockaroo.com/) para crear la data random dentro de la base de datos
3. Luego se pasó esa data a una base de datos MySQL

#### Procedimiento Scrip de Python

1. Primero se importaron las librerías de pandas y re.
```
import pandas as pd
import re
```


2. Luego se importó toda la data.
```
customers = pd.read_csv('Data/customers.csv')
invoice_items = pd.read_csv('Data/invoice_items.csv')
invoice = pd.read_csv('Data/invoice.csv')
#product_categories = pd.read_csv('Data/product_categories.csv')
product_items = pd.read_csv('Data/product_items.csv')
stores = pd.read_csv('Data/stores.csv')
```


3. Después se reemplazaron las categorías numericas por su nombre.
```
product_items['category_id'].replace({1:'Home',
                                      2:'Arms',
                                      3:'Books',
                                      4:'Industrial',
                                      5:'Grocery',
                                      6:'Computers',
                                      7:'Shoes',
                                      8:'Men Clothing',
                                      9:'Electronics',
                                      10:'Baby',
                                      11:'Toys',
                                      12:'Games',
                                      13:'Outdoors',
                                      14:'Woman Clothing',
                                      15:'Kids'}, inplace=True)
```


4. Luego se le hizo merge a todos los csv's que fueron importados para terminar con solo 1 dataframe y se reorganizaron todas las columnas.

```
df = pd.merge(stores, invoice, on='store_id')
df.drop(columns=['customer_id', 'store_id', 'invoice_date'], inplace=True)
df = pd.merge(df, invoice, on='invoice_id')
df = pd.merge(df, invoice_items, on='invoice_id')
df = pd.merge(df, product_items, on='item_id')
df = pd.merge(df, customers, on='customer_id')

df[['invoice_date', 'invoice_hour']] = df.invoice_date.str.split(" ", expand=True)
#df.drop(columns=['invoice_date'], inplace=True)

df = df[['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id', 'store_name', 'store_location_lat', 'store_location_long',
         'customer_name', 'customer_lastname', 'customer_phone', 'customer_email', 'category_id', 'product_name', 'product_price',
         'quantity', 'invoice_date', 'invoice_hour']]
```

5. Y se empezó con el primer objetivo, el cual fue encontrar las ventas hechas en un periodo especificado. Para eso primero se creó una función que suma todos los digitos de la fecha y hora declarados, luego se declaran las variables para setear el intervalo de tiempo y como final se hace un filtrado de todas las columnas para solo obtener las filas que coincidan con el intervalo declarado y se agrupan por el nombre de la tienda.

```
def find_sum(str1):
    return sum(map(int, re.findall('\d+', str1)))

fecha_inicio = '2021-01-25'
fecha_fin = '2021-01-25'
hora_inicio = '18:00:00'
hora_fin = '20:39:32'

df['invoice_date_sum'] = df['invoice_date'].apply(lambda x: sum(map(int, re.findall('\d+', x))))
df['invoice_hour_sum'] = df['invoice_hour'].apply(lambda x: sum(map(int, re.findall('\d+', x))))

df_time = df.loc[(df['invoice_date_sum']>=find_sum(fecha_inicio)) & (df['invoice_hour_sum']>=find_sum(hora_inicio)) & 
                 (df['invoice_date_sum']<=find_sum(fecha_fin)) & (df['invoice_hour_sum']<=find_sum(hora_fin))]

df_time = df_time.groupby(by='store_name').sum()
df_time.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                      'store_location_lat', 'store_location_long', 'invoice_date_sum', 'invoice_hour_sum'], inplace=True)
df_time
```

6. Y por último se empezó el segundo objetivo, el cual consitía en encontrar el top 20 productos más vendidos en total y por tienda. Para ello se agrupó y sumarizó el dataframe por el nombre de producto y se eliminaron las columnas que no aportaban valor al analisis. Después para obtener los productos más vendidos por tienda se hizo algo similar, solo que se agrupó y sumarizó primero por el nombre de la tienda y luego por el nombre de producto.

```
Productos_mas_vendidos = df.groupby(by='product_name').sum()
Productos_mas_vendidos = Productos_mas_vendidos.sort_values('quantity', ascending=False)
Productos_mas_vendidos.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                                     'store_location_lat', 'store_location_long'], inplace=True)
Productos_mas_vendidos

Productos_mas_vendidos_tienda = df.groupby(by=['store_name', 'product_name']).sum()
Productos_mas_vendidos_tienda = Productos_mas_vendidos_tienda.sort_values(['store_name', 'quantity'], ascending=False)
Productos_mas_vendidos_tienda.drop(columns=['store_id', 'customer_id', 'item_id', 'invoice_id', 'invoice_line_id',
                                     'store_location_lat', 'store_location_long'], inplace=True)
Productos_mas_vendidos_tienda
```

#### Procedimiento reporte Power BI
1. Conectarse a la base de datos desde power BI mediante MySQL WorkBench.
2. Se generó el query para extraer la información consolidada en un csv.

![Diagrama hecho manualmente](Assets/Query_powerBI.png "Diagrama hecho manualmente")

3. Se creó un gráfico de barras para mostrar el top 10 de las tiendas por un tiempo determinado.
4. Se creó un gráfico de barras para mostrar el top 10 de departamentos que más vendieron en total y por tienda.
5. Se hizo un gráfico de líneas de tendencia de ventas en un tiempo determinado en total y por tienda.